In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

path = '/kaggle/input/tabular-playground-series-jan-2022/'

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv(path + 'train.csv', index_col=0, parse_dates=['date']).set_index('date', drop=False)
test_df = pd.read_csv(path + 'test.csv', index_col=0, parse_dates=['date']).set_index('date', drop=False)

In [ ]:
train_df.head()

In [ ]:
train_df['year'] = train_df.date.dt.year
# train_df['num_sold'] = np.log(train_df.num_sold)
for col in ['country', 'store', 'product']:
    train_df[col] = train_df[col].str.replace(' ', '_')

# Is there a trend?

In [ ]:
yearly = train_df.groupby(['year', 'store', 'product', 'country']).sum()[['num_sold']].reset_index()

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(yearly.groupby(['country', 'store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    ax.set_title(combi)
    ax.set_ylim(df.num_sold.min(), df.num_sold.max())
    ax.set_ylabel('log sales')
    ax.set_xticks(range(2015, 2019))
    ax.plot(range(2015, 2019), df.num_sold)
plt.suptitle('yearly sold in 2015-2018')
plt.tight_layout()
plt.show()

In [ ]:
result = pd.DataFrame(columns=['coef', 'p_value'])
dp = DeterministicProcess(index=range(4), order=1, constant=True)
for i, (combi, df) in enumerate(yearly.groupby(['country', 'store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    X = dp.in_sample()
    y = np.array(df.num_sold)
    res = sm.OLS(y, X).fit()
    ans = [res.params[1], res.pvalues[1]]
    result.loc[str(combi)] = ans
result

From the above p value, we see that Finland and Sweden trends are significant while Norway's is not. The Kaggle discussion says that it has a correlation with GDP so we would like to see if trends still exist with GDP regression.

In [ ]:
gdp_df = pd.DataFrame([[234.440, 385.802, 505.104], [240.608, 368.827, 515.655], [255.017, 398.394, 541.019], [275.580, 437.000, 555.455], [268.782, 405.510, 533.880]], 
                   index=[2015, 2016, 2017, 2018, 2019], columns=['Finland', 'Norway', 'Sweden'])

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(yearly.groupby(['country', 'store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    country = combi[0]
    gdp = np.log(gdp_df.loc[gdp_df.index[:-1], country])
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
#     ax.set_ylim(df.num_sold.min(), df.num_sold.max())
    ax.set_xticks(range(2015, 2019))
    ax.plot(range(2015, 2019), df.num_sold, color='blue', label='log sales')
    ax.plot(range(2015, 2019), gdp / gdp.iloc[0] * df.num_sold.iloc[0], color='orange', label='log gdp')
    ax.legend()
    ax.set_title(f"Yearly sales for {combi}")
    ax.xaxis.set_major_locator(MaxNLocator(integer=True)) 
plt.suptitle('yearly sold in 2015-2018')
plt.tight_layout()
plt.show()

In [ ]:
result = pd.DataFrame(columns=['coef', 'p_value'])
dp = DeterministicProcess(index=range(4), order=1, constant=True)
for i, (combi, df) in enumerate(yearly.groupby(['country', 'store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    country = combi[0]
    gdp = np.log(gdp_df.loc[gdp_df.index[:-1], country])
#     X = dp.in_sample()
    y = np.array(df.num_sold)
    res = sm.OLS(y, sm.add_constant(gdp)).fit()
    ans = [res.params[1], res.pvalues[1]]
    result.loc[str(combi)] = ans
result

From the p value, we can see GDP plays an important role in sales. We then take a look at the residuals and see if residuals still have a trend.

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(yearly.groupby(['country', 'store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    country = combi[0]
    gdp = np.log(gdp_df.loc[gdp_df.index[:-1], country])
#     X = dp.in_sample()
    y = np.array(df.num_sold)
    res = sm.OLS(y, sm.add_constant(gdp)).fit()
    residual = y - res.predict(sm.add_constant(gdp))
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    ax.set_xticks(range(2015, 2019))
    ax.plot(range(2015, 2019), residual)
    ax.set_title(combi)
plt.suptitle('residuals in 2015-2018')
plt.tight_layout()
plt.show()

In [ ]:
result = pd.DataFrame(columns=['coef', 'p_value'])
dp = DeterministicProcess(index=range(4), order=1, constant=True)
for i, (combi, df) in enumerate(yearly.groupby(['country', 'store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    country = combi[0]
    gdp = np.log(gdp_df.loc[gdp_df.index[:-1], country])
    X = dp.in_sample()
    y = np.array(df.num_sold)
    res = sm.OLS(y, sm.add_constant(gdp)).fit()
    residual = y - res.predict(sm.add_constant(gdp))
    res = sm.OLS(np.array(residual), X).fit()
    ans = [res.params[1], res.pvalues[1]]
    result.loc[str(combi)] = ans
result

Now we can see the huge p value shows there is no trend in any combination after partialling out GDP.

# Different Sales between Countries

We would like to see if there is any difference of sales principles between countries. 

## Finland vs Norway

In [ ]:
df1 = train_df.query('country == "Finland" | country == "Norway"')

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax = plt.subplot(6, 2, i * 2 + 1, ymargin=0.5)
#     ax.set_xticks(df.index.unique())
    ax.plot(df.index.unique(), df[df.country == 'Finland'].num_sold, label='Finland')
    ax.plot(df.index.unique(), df[df.country == 'Norway'].num_sold, label='Norway')
    ax.set_title(combi)
    ax2 = plt.subplot(6, 2, i * 2 + 2, ymargin=0.5)
    differ = df[df.country == 'Finland'].num_sold - df[df.country == 'Norway'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = differ.rolling(10, center=True).mean()
#     ax2.set_xticks(df.index.unique())
    ax2.plot(df.index.unique(), differ)
    ax2.set_title(f"{combi} (with 10 day moving average)")
plt.suptitle('Finland - Norway in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax2 = plt.subplot(6, 1, i + 1, ymargin=0.5)
    differ = df[df.country == 'Finland'].num_sold - df[df.country == 'Norway'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ).sort_values(ascending=False)
    differ = differ[differ > 2]
    ax2.bar(differ.index.astype(str), differ)
    ax2.set_title(f"{combi}")
    ax2.tick_params(rotation=75)
plt.suptitle('Finland - Norway in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

To see them more clearly, we draw a plot about the outliers

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax = plt.subplot(6, 1, i + 1, ymargin=0.5)
#     ax.set_xticks(df.index.unique())
    differ = df[df.country == 'Finland'].num_sold - df[df.country == 'Norway'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ)
    differ -= 2
    differ[differ < 0] = 0
#     differ = differ.rolling(10, center=True).mean()
#     ax2.set_xticks(df.index.unique())
    ax.plot(df.index.unique(), differ)
    ax.set_title(f"{combi}")
plt.suptitle('Finland - Norway in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

Sum them up and we get

In [ ]:
plt.figure(figsize=(18, 6))
res = None
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    differ = df[df.country == 'Finland'].num_sold - df[df.country == 'Norway'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ)
#     differ -= 2
    differ[differ < 0] = 0
    if res is None:
        res = differ
    else:
        res += differ
res[res < 0] = 0
dates = sorted(res.index.strftime('%m-%d').unique())
for year in range(2015, 2019):
    data = res[res.index.year == year]
    data.index = data.index.strftime('%m-%d')
    plt.plot(dates, data.reindex(dates).fillna(0), label=year)
plt.xticks(dates[::20])
plt.ylabel('Difference')
plt.xlabel('Time')
plt.suptitle('Finland - Norway in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

In conclusion, the difference between Finland and Norway falls on the date of the followings:

1. Around Jan 10.

2. End of May.

3. End of June.

4. Beginning of November.

5. Beginning of December.

## Sweden vs Norway

In [ ]:
df1 = train_df.query('country == "Sweden" | country == "Norway"')

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax = plt.subplot(6, 2, i * 2 + 1, ymargin=0.5)
#     ax.set_xticks(df.index.unique())
    ax.plot(df.index.unique(), df[df.country == 'Sweden'].num_sold, label='Sweden')
    ax.plot(df.index.unique(), df[df.country == 'Norway'].num_sold, label='Norway')
    ax.set_title(combi)
    ax2 = plt.subplot(6, 2, i * 2 + 2, ymargin=0.5)
    differ = df[df.country == 'Sweden'].num_sold - df[df.country == 'Norway'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = differ.rolling(10, center=True).mean()
#     ax2.set_xticks(df.index.unique())
    ax2.plot(df.index.unique(), differ)
    ax2.set_title(f"{combi} (with 10 day moving average)")
plt.suptitle('Sweden - Norway in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax2 = plt.subplot(6, 1, i + 1, ymargin=0.5)
    differ = df[df.country == 'Sweden'].num_sold - df[df.country == 'Norway'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ).sort_values(ascending=False)
    differ = differ[differ > 2]
    ax2.bar(differ.index.astype(str), differ)
    ax2.set_title(f"{combi}")
    ax2.tick_params(rotation=75)
plt.suptitle('Sweden - Norway in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

To see them more clearly, we draw a plot about the outliers

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax = plt.subplot(6, 1, i + 1, ymargin=0.5)
#     ax.set_xticks(df.index.unique())
    differ = df[df.country == 'Sweden'].num_sold - df[df.country == 'Norway'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ)
    differ -= 2
    differ[differ < 0] = 0
#     differ = differ.rolling(10, center=True).mean()
#     ax2.set_xticks(df.index.unique())
    ax.plot(df.index.unique(), differ)
    ax.set_title(f"{combi}")
plt.suptitle('Sweden - Norway in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

Sum them up and we get

In [ ]:
plt.figure(figsize=(18, 6))
res = None
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    differ = df[df.country == 'Sweden'].num_sold - df[df.country == 'Norway'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ)
    differ -= 0
    differ[differ < 0] = 0
    if res is None:
        res = differ
    else:
        res += differ
res[res < 0] = 0
dates = sorted(res.index.strftime('%m-%d').unique())
for year in range(2015, 2019):
    data = res[res.index.year == year]
    data.index = data.index.strftime('%m-%d')
    plt.plot(dates, data.reindex(dates).fillna(0), label=year)
plt.xticks(dates[::20])
plt.suptitle('Sweden - Norway in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

In conclusion, the difference between Finland and Norway falls on the date of the followings:

1. Around Jan 10.

2. Beginning of April.

3. End of May.

4. End of June.

5. Around August 5.

## Finland Vs Sweden

In [ ]:
df1 = train_df.query('country == "Finland" | country == "Sweden"')

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax = plt.subplot(6, 2, i * 2 + 1, ymargin=0.5)
#     ax.set_xticks(df.index.unique())
    ax.plot(df.index.unique(), df[df.country == 'Finland'].num_sold, label='Sweden')
    ax.plot(df.index.unique(), df[df.country == 'Sweden'].num_sold, label='Norway')
    ax.set_title(combi)
    ax2 = plt.subplot(6, 2, i * 2 + 2, ymargin=0.5)
    differ = df[df.country == 'Finland'].num_sold - df[df.country == 'Sweden'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = differ.rolling(10, center=True).mean()
#     ax2.set_xticks(df.index.unique())
    ax2.plot(df.index.unique(), differ)
    ax2.set_title(f"{combi} (with 10 day moving average)")
plt.suptitle('Finland - Sweden in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax2 = plt.subplot(6, 1, i + 1, ymargin=0.5)
    differ = df[df.country == 'Finland'].num_sold - df[df.country == 'Sweden'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ).sort_values(ascending=False)
    differ = differ[differ > 2]
    ax2.bar(differ.index.astype(str), differ)
    ax2.set_title(f"{combi}")
    ax2.tick_params(rotation=75)
plt.suptitle('Finland - Sweden in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

To see them more clearly, we draw a plot about the outliers

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax = plt.subplot(6, 1, i + 1, ymargin=0.5)
#     ax.set_xticks(df.index.unique())
    differ = df[df.country == 'Finland'].num_sold - df[df.country == 'Sweden'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ)
    differ -= 2
    differ[differ < 0] = 0
#     differ = differ.rolling(10, center=True).mean()
#     ax2.set_xticks(df.index.unique())
    ax.plot(df.index.unique(), differ)
    ax.set_title(f"{combi}")
plt.suptitle('Finland - Sweden in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

Sum them up and we get

In [ ]:
plt.figure(figsize=(18, 6))
res = None
for i, (combi, df) in enumerate(df1.groupby(['store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    differ = df[df.country == 'Finland'].num_sold - df[df.country == 'Sweden'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ)
    differ -= 0
    differ[differ < 0] = 0
    if res is None:
        res = differ
    else:
        res += differ
res[res < 0] = 0
dates = sorted(res.index.strftime('%m-%d').unique())
for year in range(2015, 2019):
    data = res[res.index.year == year]
    data.index = data.index.strftime('%m-%d')
    plt.plot(dates, data.reindex(dates).fillna(0), label=year)
plt.xticks(dates[::20])
plt.suptitle('Finland - Sweden in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

In conclusion, the difference between Finland and Norway falls on the date of the followings:

1. Around June 10.

2. Around December 10.

3. Beginning of Jan.

## Conclusion

The special dates for different countries are as followed:

Finland: Around June 10; around December 10; beginning of Jan.

Sweden: (None)

Norway: Easter, End of June, Around August 5.

# Difference between stores

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax = plt.subplot(9, 2, i * 2 + 1, ymargin=0.5)
#     ax.set_xticks(df.index.unique())
    ax.plot(df.index.unique(), df[df.store == 'KaggleMart'].num_sold, label='KaggleMart')
    ax.plot(df.index.unique(), df[df.store == 'KaggleRama'].num_sold, label='KaggleRama')
    ax.set_title(combi)
    ax2 = plt.subplot(9, 2, i * 2 + 2, ymargin=0.5)
    differ = df[df.store == 'KaggleMart'].num_sold - df[df.store == 'KaggleRama'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = differ.rolling(10, center=True).mean()
#     ax2.set_xticks(df.index.unique())
    ax2.plot(df.index.unique(), differ)
    ax2.set_title(f"{combi} (with 10 day moving average)")
plt.suptitle('KaggleMart - KaggleRama in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'product'])):
    df.num_sold = np.log(df.num_sold)
    ax = plt.subplot(9, 1, i + 1, ymargin=0.5)
#     ax.set_xticks(df.index.unique())
    differ = df[df.store == 'KaggleMart'].num_sold - df[df.store == 'KaggleRama'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ)
    differ -= 2
    differ[differ < 0] = 0
#     differ = differ.rolling(10, center=True).mean()
#     ax2.set_xticks(df.index.unique())
    ax.plot(df.index.unique(), differ)
    ax.set_title(f"{combi}")
plt.suptitle('KaggleMart - KaggleRama in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(18, 6))
res = None
for i, (combi, df) in enumerate(df1.groupby(['country', 'product'])):
    df.num_sold = np.log(df.num_sold)
    differ = df[df.store == 'KaggleMart'].num_sold - df[df.store == 'KaggleRama'].num_sold
    differ = (differ - differ.mean()) / differ.std()
    differ = np.abs(differ)
    differ -= 0
    differ[differ < 0] = 0
    if res is None:
        res = differ
    else:
        res += differ
res[res < 0] = 0
dates = sorted(res.index.strftime('%m-%d').unique())
for year in range(2015, 2019):
    data = res[res.index.year == year]
    data.index = data.index.strftime('%m-%d')
    plt.plot(dates, data.reindex(dates).fillna(0), label=year)
plt.xticks(dates[::20])
plt.suptitle('KaggleMart - KaggleRama in 2015-2018')
plt.legend()
plt.tight_layout()
plt.show()



# Seasonality

In [ ]:
def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    country = combi[0]
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    ax = plot_periodogram(df.num_sold, detrend='linear', ax=ax)
    ax.set_title(f"{combi}")
plt.suptitle('Periodogram')
plt.tight_layout()
plt.show()

We can see the seasonality are much more correlated with products but not countries and stores.

In [ ]:
fourier = CalendarFourier(freq="A", order=10)  # 10 sin/cos pairs for "A"nnual seasonality

dp = DeterministicProcess(
    index=df.index.unique(),
    constant=True,               # dummy feature for bias (y-intercept)
    order=0,                     # trend (order 1 means linear)
    seasonal=True,               # weekly seasonality (indicators)
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True,                   # drop terms to avoid collinearity
)

X = dp.in_sample()
X['year'] = df.date.dt.year

In [ ]:
plt.figure(figsize=(18, 12 * 6))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    country = combi[0]
    gdp = np.log(gdp_df.loc[gdp_df.index[:-1], country])
    X_df = X.merge(gdp, left_on='year', right_index=True)
    X_df.drop(columns=['year'], inplace=True)
    model = LinearRegression(fit_intercept=False)
    model.fit(X_df, df.num_sold)
    ax = plt.subplot(18, 1, i+1, ymargin=0.5)
    ax.plot(df.num_sold, label='real sales')
    ax.plot(df.num_sold.index, model.predict(X_df), label='fit sales')
    ax.set_title(f"{combi}")
    ax.legend()
plt.suptitle('Sales')
plt.tight_layout()
plt.show()

The fit shows that the model captures the seasonality of the data but there are some spikes that the model cannot fit well. Therefore, the next step we should take a look at is these special dates.

# Special Dates

We firstly check on which dates the model is not fitting well. Before finding dates, we should get a threshold for filtering the error, here I use a simple method that is to use boxplot to find the outliers and regard them as representatives.

In [ ]:
plt.figure(figsize=(18, 12))
boxes = pd.DataFrame()
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    country = combi[0]
    gdp = np.log(gdp_df.loc[gdp_df.index[:-1], country])
    X_df = X.merge(gdp, left_on='year', right_index=True)
    X_df.drop(columns=['year'], inplace=True)
    model = LinearRegression(fit_intercept=False)
    model.fit(X_df, df.num_sold)
    residual = np.abs(df.num_sold - np.array(model.predict(X_df)))
    residual = residual.to_frame()
    residual = residual.sort_values(by='num_sold', ascending=False)
    boxes[combi] = residual
plt.xticks(rotation='vertical')
plt.boxplot(boxes, labels=boxes.columns)
plt.show()

So you can see most boxplots choose 0.17 as a threshold so we would also like to use that value for further analysis.

In [ ]:
plt.figure(figsize=(18, 12 * 6))
dates = sorted(train_df.date.dt.strftime('%m-%d').unique())
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    df.num_sold = np.log(df.num_sold)
    country = combi[0]
    gdp = np.log(gdp_df.loc[gdp_df.index[:-1], country])
    X_df = X.merge(gdp, left_on='year', right_index=True)
    X_df.drop(columns=['year'], inplace=True)
    model = LinearRegression(fit_intercept=False)
    model.fit(X_df, df.num_sold)
    residual = np.abs(df.num_sold - np.array(model.predict(X_df)))
    ax = plt.subplot(18, 1, i + 1, ymargin=0.5)
    for year in range(2015, 2019):
        data = residual[residual.index.year == year]
        data.index = data.index.strftime('%m-%d')
        ax.plot(dates, data.reindex(dates).fillna(0), label=year)
    ax.set_xticks(dates[::20])
#     print(residual.shape)
    ax.tick_params(rotation=75)
    ax.set_title(f"{combi}")
    ax.legend()
plt.tight_layout()
plt.show()

We can see that the deviations are significant at the beginning of January, Easters and December. In the next section when we build models, we would like to specify the dates we think should be a feature.

# Conclusion

1. GDP plays an important role in predict yearly sales. Partialling out GDP, there is no trend in the sales.

2. There are some special dates in which the sales of one country deviates from other two countries, specifically: 

Finland: Around June 10; around December 10; beginning of Jan.

Sweden: (None)

Norway: Easter, End of June, Around August 5.

3. There is significant seasonality in this time series. 

4. There are some time region where the sales deviates from the seasonality. They are respectively beginning of Jan, Easters, and mid and end of December. 